# Analysis of the approximation

Copyright 2025 Stephan Kulla ("Kulla")

Licensed under the Apache License, Version 2.0 (the "Apache License") and Creative Commons Attribution 4.0 International (the "CC-BY License"). You may choose either of these licenses to govern your use of this project.

You may obtain a copy of the Apache License at:
    http://www.apache.org/licenses/LICENSE-2.0

You may obtain a copy of the CC-BY License at:
    https://creativecommons.org/licenses/by/4.0/

Unless required by applicable law or agreed to in writing, software and content distributed under the Apache License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the Apache License for the specific language governing permissions and limitations under the License.

Under the CC-BY License, you are free to share and adapt the material provided you give appropriate credit, provide a link to the license, and indicate if changes were made. See the CC-BY License for full details.

Third-Party Components and Licenses:
This product also includes third-party components which are distributed under their respective licenses. Please refer to the NOTICE file for details.

## Utility functions and imports

In [1]:
%load_ext jupyter_ai_magics

In [2]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import string
import sympy as sp

from IPython.display import display, Markdown

class LatexFormatter(string.Formatter):
    def format_field(self, value, format_spec):
        if format_spec == 'tex':
            return sp.printing.latex(value)
        if format_spec == '$' or format_spec == 'inline':
            return "$" + sp.printing.latex(value) + "$"
        elif format_spec == '$$' or format_spec == 'block':
            return "$$" + sp.printing.latex(value) + "$$"
        else:
            return super().format_field(value, format_spec)

def disp_md(template, **kwargs):
    display(Markdown(LatexFormatter().format(template, **kwargs)))

## Thesis: Approximation is always the legendre approximation

### Case for integrals in the range $[-1, 1]$

In [20]:
from functools import partial

from sympy.abc import x,n
from src.approximation import matrix_of, approximation
from src.c_space import vectors, integral_forms, supremum_norm, nth_legendre_approximation

dict_polynomials = {
    "x**n": x**n,
    "x**n/n!": x**n/sp.factorial(n),
    "1/n! (1-x)**n": sp.S(1)/sp.factorial(n)*(1-x)**n,
    "(1-x)**n": (1-x)**n,
    "Legendre": sp.legendre(n, x),
    "Normed Legendre": sp.legendre(n, x)*sp.sqrt((2*n+1)/2)
}

for p1_name, p1 in dict_polynomials.items():
    for p2_name, p2 in dict_polynomials.items():
        for func in [sp.exp(x), sp.sin(x), x**3]:
            for n0 in range(4):
                approx_func = approximation(integral_forms(p1, n0, a=-1, b=1), vectors(p2, n0), func)
                legendre_approx = nth_legendre_approximation(func, n0)
                functions_are_the_same = supremum_norm(approx_func, legendre_approx) < 1e-10

                print(f"{functions_are_the_same}: for integral form {linear_form_name}, basis {vectors_name}, function {func} and n= {n0}")
                

True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 0
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 1
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 2
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 3
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 0
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 1
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 2
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 3
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function x**3 and n= 0
True: for integral form Integral: Normed Legendr

### Case for integrals in range $[0,1]$

In [25]:
def shifted_legendre(n):
    return sp.legendre(n, 2*x-1) * sp.sqrt(2*n+1)

def nth_shifted_legendre_approximation(f, n, x=sp.symbols("x")):
    polynomials = [shifted_legendre(i) for i in range(n + 1)]
    coefficients = [sp.integrate(f * polynomials[i], (x, 0, 1)) for i in range(n + 1)]

    return sum(coef * p for coef, p in zip(coefficients, polynomials))

for p1_name, p1 in dict_polynomials.items():
    for p2_name, p2 in dict_polynomials.items():
        for func in [sp.exp(x), sp.sin(x), x**3]:
            for n0 in range(4):
                approx_func = approximation(integral_forms(p1, n0, a=0, b=1), vectors(p2, n0), func)
                legendre_approx = nth_shifted_legendre_approximation(func, n0)
                functions_are_the_same = supremum_norm(approx_func, legendre_approx) < 1e-10

                print(f"{functions_are_the_same}: for integral form {linear_form_name}, basis {vectors_name}, function {func} and n= {n0}")

True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 0
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 1
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 2
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function exp(x) and n= 3
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 0
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 1
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 2
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function sin(x) and n= 3
True: for integral form Integral: Normed Legendre, basis Basis: Normed Legendre, function x**3 and n= 0
True: for integral form Integral: Normed Legendr